In [ ]:
# Importing libarries 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout 
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split 
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, classification_report
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
#Accesing to the respective Data Set

df_input = pd.read_csv('input.csv')
df_loan = pd.read_csv('loan_data.csv')
df_output = pd.read_csv('output.csv')

df_loan

In [ ]:
# Checking for nul values, data types ana NaN 

df_loan.isnull().sum()

df_loan.dtypes

In [ ]:
# converting our column on numeric 

df_loan = pd.get_dummies(df_loan, columns=['purpose'], drop_first=True)

df_loan.dtypes

In [ ]:
# histo Plot 

plt.figure(figsize=(10,6))
sns.histplot(df_loan['fico'], bins=15, kde=True)
plt.title("FICO Score Distribution")
plt.show()


In [ ]:
sns.boxplot(x='not.fully.paid', y='fico', data=df_loan)
plt.title('FICO Score vs Loan Default')
plt.show()

In [ ]:
sns.countplot(x='not.fully.paid', hue='purpose_credit_card', data=df_loan)
plt.title("Loan Default by Purpose: Credit Card")
plt.show()


In [ ]:
# Compute the correlation matrix
corr_matrix = df_loan.corr()

# Set the figure size and create the heatmap
plt.figure(figsize=(16, 12))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title("Correlation Heatmap of Loan Data")
plt.show()


In [ ]:
# Looking the amounrt of values that our target has 
df_loan['not.fully.paid'].value_counts()

In [ ]:
# Storagering the numeric data that need to be scalet before use XGBClassifier
numeric_cols = ['int.rate', 'installment', 'log.annual.inc', 'dti',
                'fico', 'days.with.cr.line', 'revol.bal', 'revol.util',
                'inq.last.6mths'] #-----> this codel will continue after 
                                  #the stopbrake that the following celles. 

In [ ]:
# def Scalar model 
scaler = StandardScaler() # Scalar Model

df_loan[numeric_cols] = scaler.fit_transform(df_loan[numeric_cols]) # Apply scaler only to numeric columns
print(f'Numeric Futers Scaler:\n{df_loan[numeric_cols]}')


In [ ]:
# dif our target and futures
X = df_loan.drop('not.fully.paid', axis=1)
y = df_loan['not.fully.paid']

In [ ]:
# Spliting our data 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# Buildong our class weights to handle imbalance
weights = compute_class_weight(class_weight='balanced',
                               classes=np.unique(y_train),
                               y=y_train)

# Compute class weights to handle imbalance
class_weight_dict = {0: weights[0], 1: weights[1]}


print(f"Computed class weights: {class_weight}")

In [ ]:
# # Building the architecture of the Neural Network

model = Sequential()

# Input layer + first hidden layer
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.3))

# Second hidden layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# Therd 
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Output Layer 
model.add(Dense(1, activation='sigmoid'))

# Compile Model 

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['Recall'])  # Track recall directly

print(model.summary())



In [ ]:
# Fiting our neural network
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    class_weight=class_weight_dict,
                    validation_data=(X_test, y_test),\
                    )

In [ ]:
# Get the predicted probabilities for the test set, flatten the array,
# then apply a threshold of 0.4 to convert probabilities to binary class predictions (0 or 1)
y_probs = model.predict(X_test).ravel()
y_pred = (y_probs>= 0.4).astype(int)

In [ ]:
print(f'Confussin Matrix:\n', confusion_matrix(y_test,y_pred))
print(f'Classification Report:\n',classification_report(y_test, y_pred))

In [ ]:
# Loop through different class weight values for the minority class to test their effect on model performance.
# For each weight, update the class_weight dictionary, retrain the model, and print the classification report.

for weight in [4, 5, 6]:
    print(f"\nTesting with class_weight = {{0:1, 1:{weight}}}")

    class_weight_dict = {0:1, 1:weight}

    model.fit(X_train, y_train,
              epochs=30,
              batch_size=32,
              class_weight=class_weight_dict,
              verbose=0) 

    print(classification_report(y_test, y_pred))

In [ ]:
# Test different probability thresholds to see how they affect classification performance.
# For each threshold, convert predicted probabilities to binary predictions and print the classification report.

thresholds = [0.35, 0.4, 0.45]

for thresh in thresholds:
    y_pred = (y_probs >= thresh).astype(int)
    print(f"\nTesting with threshold = {thresh}")
    print(classification_report(y_test, y_pred))

    print(f'\n')

In [ ]:
# Set up early stopping to halt training when the validation recall stops improving for 5 epochs,
# and restore the best model weights observed during training.

earlystop = EarlyStopping(monitor='val_record', patience=5, restore_best_weights=True,mode='max')

In [ ]:
# Train the model with early stopping, using class weights to handle imbalance,
# and validate on the test set after each epoch. Training will stop early if validation recall does not improve for 5 epochs.

history = model.fit(X_train, y_train,
                    epochs=15,
                    batch_size=32,
                    class_weight=class_weight_dict,
                    validation_data=(X_test, y_test),\
                    callbacks=[earlystop]
                    )

In [ ]:
# Trying to get the prediction back afetr adding early stop tou our {history}
y_probs = model.predict(X_test).ravel()
y_pred = (y_probs >= 0.4).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# Define thresholds to test
thresholds = [0.45, 0.5, 0.55]

# Get predicted probabilities from the last trained model
y_probs = model.predict(X_test).ravel()

for thresh in thresholds:
    y_pred = (y_probs >= thresh).astype(int)
    print(f"\n Testing with threshold = {thresh}")
    print(classification_report(y_test, y_pred))


In [ ]:
imbalance_ratio = sum(y_train == 0)/sum(y_train == 1)
print(f'Imbalance Ratio (scale_pos_weight):\n {imbalance_ratio}')

In [ ]:
# # Initialize the XGBoost classifier with parameters to handle class imbalance and set evaluation metric.

model = XGBClassifier(
    scale_per_weight = imbalance_ratio,
    use_label_incoder = False, 
    eval_metric = 'logloss',
    random_state=42
)

In [ ]:
# Fiting model 

model.fit(X_train, y_train)

In [ ]:
# Checking the predictions 

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# Define the hyperparameter grid for XGBoost to search over during grid search.

param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'n_stimqtors': [100 , 200],
    'subsample': [0.8, 1]
}

In [ ]:
# Create an XGBoost classifier with class imbalance handling and specific evaluation metric,

xgb_model = XGBClassifier(
    scale_pos_weight=imbalance_ratio,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

In [ ]:
# Set up grid search to find the best hyperparameters for the XGBoost model.

grid = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    verbose=1,
    n_jobs=-1
)

In [ ]:
# Run grid search
grid.fit(X_train, y_train)

# Best parameters
print(f"\nBest parameters: {grid.best_params_}")

# Best model
best_model = grid.best_estimator_

# Predict and evaluate
y_pred = best_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# Apply SMOTE to balance the classes in the training data by oversampling the minority class.

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(f"Before SMOTE: {np.bincount(y_train)}")
print(f"After SMOTE: {np.bincount(y_train_res)}")

In [ ]:
y_pred = best_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Project Summary Notes

In this project, we aimed to predict loan defaults using deep learning and classical machine learning models.

The main challenge was the strong class imbalance, with around 80% non-defaults and 20% defaults.

To address this, we tested:

- Neural networks (Sequential, Keras) with:
  - Class weights
  - Dropout and L2 regularization
  - Early stopping
  - Threshold tuning

- XGBoost and RandomForest with:
  - `scale_pos_weight` parameter
  - Manual hyperparameter tuning
  - GridSearchCV for automated tuning

- Synthetic oversampling using SMOTE

- Ensemble methods (EnsembleVoteClassifier) combining XGBoost, RandomForest, and LogisticRegression

**Observations:**

- Neural networks overfit easily, even with regularization.
- Tree-based models and ensembles improved robustness but still struggled to lift recall on the minority class.
- The extreme class imbalance remained the key limitation affecting performance.

**Future suggestions:**

- Explore more advanced imbalance strategies like ADASYN or cost-sensitive learning.
- Consider feature engineering or domain-specific insights to improve model separation.
